In [37]:
import math

import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("data/full_data.csv")
df.shape

(18218, 4)

# Corrections

In [72]:
df = df.replace('*', np.nan)

# don't know what SPECIES tag is, so we are removing it for now
df = df.replace('SPECIES', np.nan)

In [48]:
df.head()

,sentence,word,hierarchical,non-hierarchical
0,0,active,DEFINIENDUM,NaN
1,0,cave,DEFINIENDUM,NaN
2,0,:,NaN,NaN
3,0,Cave,GENUS,NaN
4,0,containing,NaN,HAS_FUNCTION


In [18]:
df.isna().sum().sort_values()[:10]

sentence                0
word                    1
non-hierarchical     7936
hierarchical        15107
dtype: int64

# Check number of sentences

In [52]:
# number of different sentences
df['sentence'].nunique()

745

In [73]:
# number of sentences with each hierarchical tag
df.groupby('hierarchical')['sentence'].nunique().sort_values(ascending=False)

hierarchical
DEFINIENDUM    741
GENUS          614
DEFINITOR      275
Name: sentence, dtype: int64

In [74]:
# number of sentences with each non-hierarchical tag
df.groupby('non-hierarchical')['sentence'].nunique().sort_values(ascending=False)

non-hierarchical
HAS_FORM              287
HAS_LOCATION          282
HAS_CAUSE             221
COMPOSITION_MEDIUM    157
HAS_FUNCTION          101
HAS_SIZE              100
CONTAINS               75
HAS_ATTRIBUTE          56
DEFINED_AS             48
HAS_RESULT             20
HAS_POSITION           18
OCCURS_IN_TIME         16
MEASURES               10
STUDIES                 7
AFFECTS                 2
OCCURS_IN_MEDIUM        1
Name: sentence, dtype: int64

In [75]:
# find sentences without definiendum
idx = df.groupby('sentence')['hierarchical'].agg(lambda x: 'DEFINIENDUM' not in list(x))
idx = idx[idx==True].index
df[df['sentence'].isin(idx)]

,sentence,word,hierarchical,non-hierarchical


In [76]:
# remove sentences without definiendum
df = df[~df['sentence'].isin(idx)]

In [77]:
df

,sentence,word,hierarchical,non-hierarchical
0,0,active,DEFINIENDUM,NaN
1,0,cave,DEFINIENDUM,NaN
2,0,:,NaN,NaN
3,0,Cave,GENUS,NaN
4,0,containing,NaN,HAS_FUNCTION
...,...,...,...,...
18213,744,are,NaN,HAS_CAUSE
18214,744,shaped,NaN,HAS_CAUSE
18215,744,by,NaN,HAS_CAUSE
18216,744,eddies,NaN,HAS_CAUSE


# Prepare dataframe for training

One data frame will have hierarchical tags (DEFINIENDUM , GENUS, DEFINITOR),
the other frame will have non-hierarchical tags with at least 200 sentences (HAS_FORM, HAS_CAUSE, HAS_LOCATION) + DEFINIENDUM
Two frames are needed, because sometimes GENUS and some hierarchical tag are coinciding.

In [ ]:
df_h =